# Assignment 1.

In [1]:
import numpy as np
import scipy.linalg as la
import itertools as it
import time
import pylab as pl
from mpl_toolkits.mplot3d import Axes3D

## 1.a loss function `mean_absolute_error`

In [2]:
def mean_absolute_error(y_true, y_pred):
    ''' your code here '''
    if not isinstance(y_true,np.ndarray) and isinstance(y_pred, np.ndarray):
        print(f"Both inputs must be np-array.")
        
    return sum(abs(y_true - y_pred)) / len(y_pred)


## 1.b Cross-Validation function `cv`
Inputs:
- X: matrix
- y: vertor
- method: object
- parameters: dictionary
- loss_function: function
- nfolds: int
- nrepetitions: int

Output: `method` object includs functions `fit(X, y)` and `predict(X)`
- Output is a trained model

In [3]:
def noisysincfunction(N, noise):
    ''' noisysincfunction - generate data from the "noisy sinc function"
        % usage
        %     [X, Y] = noisysincfunction(N, noise)
        %
        % input
        %     N: number of data points
        %     noise: standard variation of the noise
        %
        % output
        %     X: (1, N)-matrix uniformly sampled in -2pi, pi
        %     Y: (1, N)-matrix equal to sinc(X) + noise
        %
        % description
        %     Generates N points from the noisy sinc function
        %
        %        X ~ uniformly in [-2pi, pi]
        %        Y = sinc(X) + eps, eps ~ Normal(0, noise.^2)
        %
        % author
        %     Mikio Braun
    '''
    X = np.sort(2 * np.pi * np.random.rand(1, N) ) - np.pi
    Y = np.sinc(X) + noise * np.random.randn(1, N)
    return X.reshape(-1, 1), Y.flatten()

In [4]:
class KFold:
    def __init__(self, n_splits=5):
        self.n_splits = n_splits

    def split(self, X):
        n_samples = len(X)
        indices = np.arange(n_samples)

        rng = np.random.default_rng(True)
        rng.shuffle(indices)
        
        fold_sizes = np.full(self.n_splits, n_samples // self.n_splits, dtype=int)
        fold_sizes[:n_samples % self.n_splits] += 1
        current = 0
        splits = []
        
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            test_idx = indices[start:stop]
            train_idx = np.concatenate((indices[:start], indices[stop:]))
            splits.append((train_idx, test_idx))
            current = stop
        
        return splits

kf = KFold(n_splits=3)
X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])

splits = kf.split(X)
for i, (train_index, test_index) in enumerate(splits):
    print(f"Fold {i + 1}")
    print("TRAIN:", train_index, "TEST:", test_index)

Fold 1
TRAIN: [4 2 5 8 6 3] TEST: [7 0 1]
Fold 2
TRAIN: [7 0 1 8 6 3] TEST: [4 2 5]
Fold 3
TRAIN: [7 0 1 4 2 5] TEST: [8 6 3]


In [8]:
import itertools
import numpy as np
def cv(X, y, method, params, loss_function=mean_absolute_error, nfolds=10, nrepetitions=5):
    ''' your header here!
    '''
    best_loss = float('inf')
    best_params = None
    best_model = None
    kf = KFold(n_splits=nfolds)
    
    # all_param_combinations = list(itertools.product(*parameters.values()))
    
    # all_param_combinations = np.array(list(itertools.product(params['regularization'], params['kernel'])))
    param_keys = list(params.keys())
    all_param_combinations = list(itertools.product(*(params[key] for key in param_keys)))
    
    total_iterations = len(all_param_combinations) * nrepetitions * nfolds
    iteration = 0
    start_time = time.time()

    for param_combination in all_param_combinations:
        # param_dict = dict(zip(['regularization', 'kernel'], param_combination))
        # param_dict['kernelparameter'] = params['kernelparameter']
        param_dict = dict(zip(param_keys, param_combination))


        total_loss = 0
        
        for _ in range(nrepetitions):
            fold_losses = []
            for train_index, val_index in kf.split(X):
                X_train, X_val = X[train_index], X[val_index]
                y_train, y_val = y[train_index], y[val_index]
                
                model = method(**param_dict)
                model.fit(X_train, y_train)
                y_pred = model.predict(X_val)
                
                fold_loss = loss_function(y_val, y_pred)
                fold_losses.append(fold_loss)

                # Update iteration and report progress
                iteration += 1
                elapsed_time = time.time() - start_time
                remaining_time = (elapsed_time / iteration) * (total_iterations - iteration)
                print(f'Progress: {iteration}/{total_iterations} - '
                      f'Elapsed Time: {elapsed_time:.2f}s - '
                      f'Remaining Time: {remaining_time:.2f}s', end='\r')
                
            total_loss += np.mean(fold_losses)
        
        avg_loss = total_loss / nrepetitions
        
        if avg_loss < best_loss:
            best_loss = avg_loss
            best_params = param_dict
            best_model = method(**best_params)
            best_model.fit(X, y)
    
    best_model.cvloss = best_loss
    return best_model


In [9]:
class krr():
    ''' Kernel Ridge Regression (KRR) implements ridge regression with various kernel functions.
        This class allows specification of the kernel type, kernel parameters, and regularization strength.
    '''
    def __init__(self, kernel='linear', kernelparameter=1, regularization=0):
        self.kernel = kernel
        self.kernelparameter = kernelparameter
        self.regularization = regularization
        self.alpha = None  # Coefficient matrix for predictions
        self.X_train = None  # Store training features here
    
    def _compute_kernel(self, X1, X2):
        ''' Compute the kernel matrix between datasets X1 and X2 based on the specified kernel type. '''
        if self.kernel == 'linear':
            return np.dot(X1, X2.T)
        elif self.kernel == 'polynomial':
            return (np.dot(X1, X2.T) + 1) ** self.kernelparameter
        elif self.kernel == 'gaussian':
            sq_dists = np.sum(X1**2, axis=1).reshape(-1, 1) + np.sum(X2**2, axis=1) - 2 * np.dot(X1, X2.T)
            return np.exp(-0.5 / self.kernelparameter**2 * sq_dists)
        else:
            raise ValueError("Unsupported kernel type")

    def fit(self, X, y, kernel=False, kernelparameter=False, regularization=False):
        ''' Fit the KRR model using the provided training data and labels.
            X: Feature matrix.
            y: Target vector.
            Optionally, kernel type, kernel parameter, and regularization can be adjusted.
        '''
        if kernel is not False:
            self.kernel = kernel
        if kernelparameter is not False:
            self.kernelparameter = kernelparameter
        if regularization is not False:
            self.regularization = regularization
        
        self.X_train = X
        n = X.shape[0]
        K = self._compute_kernel(X, X)
        I = np.eye(n)
        self.alpha = np.linalg.inv(K + self.regularization * I) @ y

        return self

    def predict(self, X):
        ''' Predicts the target values for given input features using the trained KRR model.
            X: New feature matrix for which predictions are to be made.
            Raises ValueError if the model has not been fitted.
        '''
        if self.X_train is None:
            raise ValueError("Model has not been fitted yet. Please call 'fit' with appropriate data.")
        
        K_new = self._compute_kernel(X, self.X_train)
        return np.dot(K_new, self.alpha)



In [10]:
# from sklearn.kernel_ridge import KernelRidge

Xtr, Ytr = noisysincfunction(100, 0.1)
Xte = np.arange( -np.pi, np.pi, 0.01 ).reshape(-1, 1)

params = { 'kernel': ['linear'], 'kernelparameter': np.logspace(-4,4,20),
                'regularization': np.logspace(-2,2,10) }
cvkrr = cv(Xtr, Ytr, krr, params, nrepetitions=2)
ypred = cvkrr.predict(Xte)
print('Regularization range: 10**-4 .. 10**4')
print('Gaussian kernel parameter: ', cvkrr.kernelparameter)
print('Regularization paramter: ', cvkrr.regularization)

Regularization range: 10**-4 .. 10**4.43s - Remaining Time: 0.00s
Gaussian kernel parameter:  0.0001
Regularization paramter:  100.0


In [ ]:
import itertools

params = { 'kernel': ['gaussian'], 'kernelparameter': np.logspace(-4,4,20),
        'alpha': np.logspace(-2,2,10) }

all_param_combinations = np.array(list(itertools.product(params['kernel'], params['alpha'])))
# all_param_combinations + params['kernelparameter']
for param_combination in all_param_combinations:
        param_dict = dict(zip(['kernel', 'alpha'], param_combination))
        print(param_dict)
        param_dict['kernelparameter'] = params['kernelparameter']
        
        total_loss = 0

{'kernel': 'gaussian', 'alpha': '0.01'}
{'kernel': 'gaussian', 'alpha': '0.027825594022071243'}
{'kernel': 'gaussian', 'alpha': '0.0774263682681127'}
{'kernel': 'gaussian', 'alpha': '0.21544346900318834'}
{'kernel': 'gaussian', 'alpha': '0.5994842503189409'}
{'kernel': 'gaussian', 'alpha': '1.6681005372000592'}
{'kernel': 'gaussian', 'alpha': '4.6415888336127775'}
{'kernel': 'gaussian', 'alpha': '12.915496650148826'}
{'kernel': 'gaussian', 'alpha': '35.93813663804626'}
{'kernel': 'gaussian', 'alpha': '100.0'}


In [ ]:
list(itertools.product(*params['alpha'].values(), *params['kernel']))

[('gaussian', 0.0001, 0.01),
 ('gaussian', 0.0001, 0.027825594022071243),
 ('gaussian', 0.0001, 0.0774263682681127),
 ('gaussian', 0.0001, 0.21544346900318834),
 ('gaussian', 0.0001, 0.5994842503189409),
 ('gaussian', 0.0001, 1.6681005372000592),
 ('gaussian', 0.0001, 4.6415888336127775),
 ('gaussian', 0.0001, 12.915496650148826),
 ('gaussian', 0.0001, 35.93813663804626),
 ('gaussian', 0.0001, 100.0),
 ('gaussian', 0.00026366508987303583, 0.01),
 ('gaussian', 0.00026366508987303583, 0.027825594022071243),
 ('gaussian', 0.00026366508987303583, 0.0774263682681127),
 ('gaussian', 0.00026366508987303583, 0.21544346900318834),
 ('gaussian', 0.00026366508987303583, 0.5994842503189409),
 ('gaussian', 0.00026366508987303583, 1.6681005372000592),
 ('gaussian', 0.00026366508987303583, 4.6415888336127775),
 ('gaussian', 0.00026366508987303583, 12.915496650148826),
 ('gaussian', 0.00026366508987303583, 35.93813663804626),
 ('gaussian', 0.00026366508987303583, 100.0),
 ('gaussian', 0.00069519279617